In [ ]:
# import library
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
print(tf.__version__)
import zipfile,os
import random
import shutil
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from google.colab import files
from keras.preprocessing import image

2.6.0


In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir(r'/content/drive/My Drive/Tugas/satria-data')


In [ ]:
#!unzip "data.zip"

In [ ]:
base_dir = '/content/drive/My Drive/Tugas/satria-data/data'
train = os.path.join(base_dir, 'Train_gender')
test = os.path.join(base_dir, 'Testing')

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest',
                    validation_split=0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train,
        target_size=(200, 260),
        batch_size=20,
        class_mode='binary',
        subset='training')
 
val_generator = train_datagen.flow_from_directory(
        train,
        target_size=(200, 260),
        batch_size=20,
        class_mode='binary',
        subset='validation')

Found 1849 images belonging to 2 classes.
Found 461 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(200, 260, 3)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 258, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 128, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 126, 128)      73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 94, 124, 128)      147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 62, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 45, 60, 256)       2

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
callbacks = myCallback()

iterate_model = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator), 
      epochs=50, 
      validation_data=val_generator, 
      validation_steps=len(val_generator),  
      verbose=1,
      callbacks=[callbacks])

Epoch 1/50
93/93 [==============================] - 2246s 24s/step - loss: 0.8119 - accuracy: 0.5787 - val_loss: 0.6812 - val_accuracy: 0.5813
Epoch 2/50
93/93 [==============================] - 2118s 23s/step - loss: 0.6829 - accuracy: 0.5760 - val_loss: 0.6805 - val_accuracy: 0.5813
Epoch 3/50
93/93 [==============================] - 2111s 23s/step - loss: 0.6824 - accuracy: 0.5803 - val_loss: 0.6807 - val_accuracy: 0.5813
Epoch 4/50
93/93 [==============================] - 2107s 23s/step - loss: 0.6812 - accuracy: 0.5803 - val_loss: 0.6799 - val_accuracy: 0.5813
Epoch 5/50
93/93 [==============================] - 2109s 23s/step - loss: 0.6803 - accuracy: 0.5803 - val_loss: 0.6798 - val_accuracy: 0.5813
Epoch 6/50
93/93 [==============================] - 2112s 23s/step - loss: 0.6816 - accuracy: 0.5803 - val_loss: 0.6799 - val_accuracy: 0.5813
Epoch 7/50
93/93 [==============================] - 2124s 23s/step - loss: 0.6804 - accuracy: 0.5803 - val_loss: 0.6799 - val_accuracy: 0.5813

In [ ]:
import matplotlib.pyplot as plt
acc = iterate_model.history['accuracy']
val_acc = iterate_model.history['val_accuracy']
loss = iterate_model.history['loss']
val_loss = iterate_model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()